## Embedding a Bokeh server in a Notebook

This notebook shows how a Bokeh server application can be embedded inside a Jupyter notebook. 

In [15]:
import yaml

import pandas as pd

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook

from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature

output_notebook()

In [16]:
io_traces = None
with open("io_operations.yml", "r", encoding="utf-8") as traces:
    io_traces = yaml.load(traces, Loader=yaml.SafeLoader)
    
# Print first job trace
print(io_traces[2])


job_columns = [
    "job_id", 
    "job_status", 
    "job_submit_ts", 
    "job_end_ts", 
    "job_duration",
    "origin_runtime",
    "origin_read_bytes",
    "origin_written_bytes",
    "origin_core_used",
    "origin_mpi_procs",
    "job_sleep_time",
]

# Not all columns are used by every type of action...
action_columns = [
    "act_name",
    "act_type",
    "act_status",
    "act_start_ts",
    "act_end_ts",
    "act_duration",
    "src_storage_service",
    "src_storage_server",
    "src_storage_disk",
    "src_file_path",
    "src_file_name",
    "src_file_size_bytes",
    "dst_storage_service",
    "dst_storage_server",
    "dst_storage_disk",
    "dst_file_path",
    "dst_file_name",
    "dst_file_size_bytes",
]




pd.DataFrame(io_traces, columns=job_columns)


There are various application handlers that can be used to build up Bokeh documents. For example, there is a `ScriptHandler` that uses the code from a `.py` file to produce Bokeh documents. This is the handler that is used when we run `bokeh serve app.py`. In the notebook we can use a function to define a Bokeh application.

Here is the function `bkapp(doc)` that defines our app:

In [3]:
def bkapp(doc):
    df = sea_surface_temperature.copy()
    source = ColumnDataSource(data=df)

    plot = figure(x_axis_type='datetime', y_range=(0, 25),
                  y_axis_label='Temperature (Celsius)',
                  title="Sea Surface Temperature at 43.18, -70.43")
    plot.line('time', 'temperature', source=source)

    def callback(attr, old, new):
        if new == 0:
            data = df
        else:
            data = df.rolling('{0}D'.format(new)).mean()
        source.data = ColumnDataSource.from_df(data)

    slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))

    doc.theme = Theme(json=yaml.load("""
        attrs:
            figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))

Now we can display our application using ``show``, which will automatically create an ``Application`` that wraps ``bkapp`` using ``FunctionHandler``. The end result is that the Bokeh server will call ``bkapp`` to build new documents for every new sessions that is opened.

**Note**: If the current notebook is not displayed at the default URL, you must update the `notebook_url` parameter in the comment below to match, and pass it to `show`.

In [4]:
show(bkapp) # notebook_url="http://localhost:8888" 